# ShowCase ECG preprocessing and HRV analysis WeLoveReading

### Purpose
Purpose of this notebook is to preprocess the raw ECG data for the 'We Love Reading' study.

- Neurokit2 package with a custom pipeline will be used for preprocessing
- Segmentation is performed afterwards
- QA figures will be exported for every partipant
- The processed signal data will be exported
- HRV metrics are computed and exported


### Input / Output
- Input: `~/data/raw`
- Outputs:
  - QA visualizations: `~/reports/ECG QA`
  - Processed signal data and HRV metrics: `~/data/processed`
  
**Note**:
- Output folders are created if they do not exist
- Make sure the **filenames follows the following format**:
  -  `[condition][id]_W[wavenumber]_mc.txt` for the ECG recordings
  -  `[condition][id]_W[wavenumber]_event.txt` for the corresponding events
  -  ...where 
     -  `[condition]` is a single letter
     -  `[id]` is a number
     -  `[wavenumber]` is a number 
  -  *Example*: B40_W3_mc.txt
  -  Note that it is not case sensitive 


### Imports

In [1]:
# fmt: off
from pathlib import Path
import numpy as np
import importlib
import neurokit2 as nk
import sys
sys.path.append(str(Path().cwd().parent/'src'))
import utils.parameters as params
import utils.common as common
import utils.data_utils as data_utils
import utils.nk_pipeline as nk_pipeline
import app.analyse_we_love_reading as app 
importlib.reload(nk)
importlib.reload(nk_pipeline)
importlib.reload(params)
importlib.reload(data_utils)
importlib.reload(common)
import matplotlib.pyplot as plt
plt.ioff()  # Turn off interactive mode
import pandas as pd
# fmt:on

# <span style="color:blue">Apply the pipeline</span>

In the following, we will show how the pipeline can be used at three different levels of abstraction.

#### ShowCase 1: High-level Workflow
Showcase 1 provides a high-level overview of the workflow. To demonstrate what the pipeline does, an exemplary dyad will be used and the following workflow will be executed:
1. Defining the data files to be used and loading the ECG and event data.
2. Splitting the data into a timeseries for the mother and the child, respectively.
3. Loading (and specifying) the parameters used for segmenting the conditions and preprocessing the ECG data
4. Preprocessing the ECG signals
5. Segmenting the ECG signals
6. Calculating HRV metrics 

#### ShowCase 2: Processing a dyad
In ShowCase 2, most of the workflow shown in ShowCase 1 is encapsulated in a single function called process_dyad(), which makes the pipeline easier to use.

#### ShowCase 3: Processing all dyads
ShowCase 3 reveals the highest level of abstraction: processing and analysing all dyads using a single function called process_all_dyads().


## Define Input and Output parameters

In [4]:
WORKING_DIR = Path().cwd() # the current directory
ROOT_DIR = WORKING_DIR.parent
DATA_DIR = ROOT_DIR / 'data'
RAW_DATA_DIR = DATA_DIR / 'wlr_raw'
PROCESSED_DATA_DIR = DATA_DIR / 'processed' # where to save output data 
PROCESSED_DATA_DIR.mkdir(parents=False, exist_ok=True)
QA_REPORTS_DIR = ROOT_DIR / "reports" / 'QA' # where to save QA visualizations
QA_REPORTS_DIR.mkdir(parents=True, exist_ok=True) 

### <span style="color:red">ShowCase 1: High-level Workflow</span>

Loading the ECG and Event data from an exemplary dyad

In [6]:
ecg_filepath = RAW_DATA_DIR / "b01_w2_mc.txt"
event_filepath = RAW_DATA_DIR / "b01_w2_event.txt"
signal_event_df = data_utils.load_dyad_ecg_events(ecg_filepath, event_filepath)
signal_event_df.head()

,child_ecg,mother_ecg,event,event_description
seconds,,,,
0.000,0.029664,-0.006464,NaN,NaN
0.002,0.029662,-0.006449,NaN,NaN
0.004,0.029667,-0.006456,NaN,NaN
0.006,0.029690,-0.006499,NaN,NaN
0.008,0.029683,-0.006529,NaN,NaN


Split the dataframe into two timeseries: mother and child

In [7]:
child_series, mother_series = data_utils.split_in_child_mother_series(signal_event_df)
display(child_series.head())
display(mother_series.head())

seconds
0.000    0.029664
0.002    0.029662
0.004    0.029667
0.006    0.029690
0.008    0.029683
Name: child_ecg, dtype: float64

seconds
0.000   -0.006464
0.002   -0.006449
0.004   -0.006456
0.006   -0.006499
0.008   -0.006529
Name: mother_ecg, dtype: float64

Define the parameters used for preprocessing and segmentation

In [8]:
dyad_id, condition, wave = data_utils.extract_subject_id_condition_from_filepath(ecg_filepath)
parameters = params.base_params # load the default parameters

In [9]:
# use the function configure_segmentation_params to configure the segmentation parameters for the dyad
# use the function configure_ecg_params to configure the ECG preprocessing parameters for the mother and/or the child
segmentation_params = params.configure_segmentation_params(dyad_id, parameters)
child_params, mother_params = params.configure_ecg_params(dyad_id, segmentation_params.copy())

Preprocess the ECG data and combine it again with the event data

In [10]:
# Preprocess
child_signals_df = nk_pipeline.ecg_preprocess(child_series, child_params)
mother_signals_df = nk_pipeline.ecg_preprocess(mother_series, mother_params)

child_signals_df.head()

,ECG_Clean,ECG_Raw,ECG_R_Peaks
seconds,,,
0.000,-0.000019,0.029664,0
0.002,-0.000017,0.029662,0
0.004,-0.000015,0.029667,0
0.006,-0.000013,0.029690,0
0.008,-0.000011,0.029683,0


In [11]:
# Combine with event data
child_signal_event_df = child_signals_df.merge(signal_event_df[["event", "event_description"]], left_index=True, right_index=True, how = "left")
mother_signal_event_df = mother_signals_df.merge(signal_event_df[["event", "event_description"]], left_index=True, right_index=True, how = "left")

child_signal_event_df[~pd.isna(child_signal_event_df["event_description"])].head()

,ECG_Clean,ECG_Raw,ECG_R_Peaks,event,event_description
seconds,,,,,
79.340,-3.062538e-05,-0.030098,0,Keyboard:F1,baseline resting start
477.560,-1.487674e-04,-0.027758,0,Keyboard:F2,light switch
483.330,-2.167820e-05,-0.027525,0,Keyboard:F3,Book start
1339.472,9.833777e-07,-0.022009,0,Keyboard:F6,AKT start
1387.820,8.832925e-05,-0.021598,0,Keyboard:F4,stroop mother start


Segment the preprocessed data based on the information provided in the parameters

In [12]:
child_segments_df_list = data_utils.segment_df(child_signal_event_df, segmentation_params)
mother_segments_df_list = data_utils.segment_df(mother_signal_event_df, segmentation_params)

# The result is a list with dataframes: one dataframe per segment
print("Segment 1")
display(child_segments_df_list[0].head(3))
print("Segment 2")
display(child_segments_df_list[1].head(3))

Segment 1


,ECG_Clean,ECG_Raw,ECG_R_Peaks,event,event_description
seconds,,,,,
79.340,-0.000031,-0.030098,0,Keyboard:F1,baseline resting start
79.342,-0.000014,-0.030131,0,NaN,NaN
79.344,0.000009,-0.030160,0,NaN,NaN


Segment 2


,ECG_Clean,ECG_Raw,ECG_R_Peaks,event,event_description
seconds,,,,,
483.330,-0.000022,-0.027525,0,Keyboard:F3,Book start
483.332,-0.000021,-0.027533,0,NaN,NaN
483.334,-0.000021,-0.027527,0,NaN,NaN


Calculate HRV metrics and create QA-plots

*Note that HRV metrics are calculated in windows of e.g., 30 seconds within a segment. The duration of the analysis window is defined in the parameters.*

In [55]:
# Compute windowed HRV metrics per segment and subject
hrv_child_df, ecg_child_df = app.compute_windowed_hrv_across_segments(
    segments_df_list=child_segments_df_list,
    parameters=child_params,
    figure_output_dir=QA_REPORTS_DIR,
    data_output_dir=PROCESSED_DATA_DIR,
    subject_pair="child",
    create_qa_plots=False
)


hrv_mother_df, ecg_mother_df = app.compute_windowed_hrv_across_segments(
    segments_df_list=mother_segments_df_list,
    parameters=mother_params,
    figure_output_dir=QA_REPORTS_DIR,
    data_output_dir=PROCESSED_DATA_DIR,
    subject_pair="mother",
    create_qa_plots=False
)

display(hrv_child_df.head(3))

/Users/lukasspiess/anaconda3/envs/neuroprofile/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


,HRV_MeanNN,HRV_SDNN,HRV_SDANN1,HRV_SDNNI1,HRV_SDANN2,HRV_SDNNI2,HRV_SDANN5,HRV_SDNNI5,HRV_RMSSD,HRV_SDSD,...,HRV_pNN20,HRV_MinNN,HRV_MaxNN,HRV_HTI,HRV_TINN,start_index,stop_index,analysis_window,heart_rate_bpm,segment_name
0,512.551724,12.310561,NaN,NaN,NaN,NaN,NaN,NaN,12.377115,12.481063,...,8.620690,486.0,540.0,3.222222,46.8750,79.34,109.338,0,118.0,baseline resting start
1,507.068966,12.111850,NaN,NaN,NaN,NaN,NaN,NaN,12.599081,12.703975,...,8.620690,482.0,538.0,3.625000,46.8750,109.34,139.338,1,118.0,baseline resting start
2,511.551724,10.184564,NaN,NaN,NaN,NaN,NaN,NaN,10.529824,10.622481,...,1.724138,490.0,538.0,3.411765,39.0625,139.34,169.338,2,118.0,baseline resting start


### <span style="color:red">ShowCase 2: Process a single dyad</span>

**Below an example for how to process and analyse the data from a single dyad.**

Read all ECG datafiles in the Raw data directory

In [15]:
ecg_filepaths = np.sort(list(RAW_DATA_DIR.glob('*mc.txt')))
# [print(f) for f in ecg_filepaths];
# [print(data_utils.extract_subject_id_condition_from_filepath(f)) for f in ecg_filepaths];

Read all Event datafiles in the Raw data directory

In [16]:
event_filepaths = np.sort(list(RAW_DATA_DIR.glob('*event.txt')))
# [print(f) for f in event_filepaths];
# [print(data_utils.extract_subject_id_condition_from_filepath(f)) for f in event_filepaths];

In [17]:
file_index = 0 # whether to process the first, second, third etc. recording

# Preprocess the data, compute HRV metrics and save the output
app.process_dyad(
    ecg_filepath=ecg_filepaths[file_index],
    event_filepath=event_filepaths[file_index],
    parameters=params.base_params,
    data_output_dir=PROCESSED_DATA_DIR,
    figure_output_dir=QA_REPORTS_DIR,
    create_qa_plots=True
)

/Users/lukasspiess/anaconda3/envs/neuroprofile/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


### <span style="color:red">ShowCase 3: Process all dyads</span>

### 

**Below an example for how to process all dyads, one after another**.  
Much of the preparations needed to process a single dyad is incorporated in the function `app.process_all_dyads()`. In addition, a log file is created in `~/`

In [ ]:
app.process_all_dyads(
    raw_data_dir=RAW_DATA_DIR,
    processed_data_dir=PROCESSED_DATA_DIR,
    reports_dir=QA_REPORTS_DIR,
)